In [7]:
import warnings
import os
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow as tf
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Lambda
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D
import tensorflow.keras.backend as K

In [9]:
with open('./cropped_numpys/cropped_test_X.npy', 'rb') as f:
    arr_test_X = np.load(f)

with open('./cropped_numpys/cropped_test_X_names.npy', 'rb') as f:
    test_X_names = np.load(f)

with open('./cropped_numpys/cropped_test_Y.npy', 'rb') as f:
    arr_test_Y = np.load(f)

In [10]:
#function creating pairs
def	make_pairs(images, labels, image_names):

	pairImages = []
	pairLabels = []
	pairImagesNames = []
	uniqueClasses = np.unique(labels)

	dict_idx = {i:np.where(labels == i)[0] for i in uniqueClasses}

	for idxA in range(len(images)):
		currentImage = images[idxA]
		label = labels[idxA]
		currentImage_name = image_names[idxA]

		#positive pair
		idxB = np.random.choice(dict_idx[label])
		posImage = images[idxB]
		posImage_name = image_names[idxB]
		pairImages.append([currentImage, posImage])
		pairImagesNames.append([currentImage_name, posImage_name])
		pairLabels.append([1])

		#negative pair
		negLab = np.random.choice([i for i in dict_idx.keys() if i != label])
		negIdx = np.random.choice(dict_idx[negLab])
		negImage = images[negIdx]
		negImage_name = image_names[negIdx]
		pairImages.append([currentImage, negImage])
		pairImagesNames.append([currentImage_name, negImage_name])
		pairLabels.append([0])

	return (np.array(pairImages),np.array(pairLabels)), np.array(pairImagesNames)

In [11]:
(pairTest, labelTest), pairNamesTest = make_pairs(arr_test_X, arr_test_Y, test_X_names)

In [12]:
model_file = "./contrastive_loss_model_.h5"
model = keras.models.load_model(model_file, compile=False)

In [13]:
predictions = model.predict([pairTest[:, 0], pairTest[:, 1]])

125/125 [==============================] - 52s 378ms/step
